# 👹 Fera Formidável 4.1

> Atividade realizada em dupla: Caio Ruas (24010) e Thalles Cansi (24006)

Um novo nível de monstro aparece nas florestas mágicas de LUMI. Esta é a primeira aparição de um monstro do tipo **Fera Formidável**. Para derrotar esta fera, precisaremos utilizar nossos conhecimentos de Redes Neurais para resolver um problema de
classificação. Vamos treinar uma rede neural com dados da planta Íris [1] que é um conjunto de dados clássico em tarefas de classificação, possui 150 amostras com 4 atributos (comprimento e largura das sépalas e pétalas) e 3 classes correspondentes às espécies de íris.

![Flores de Íris](Imagens/FloresÍris.png)

<center>
Legenda 1: Diferenças entre flores de Íris [2].
</center>

Estaremos utilizando o mesmo código da aula, provido pelo professor, para construir e treinar a rede neural.

## 🔢 Valor

Esta é a classe Valor, que representa um valor numérico com suporte a diferenciação automática. Ela possui métodos para operações matemáticas, como adição, subtração, multiplicação e exponenciação, além de métodos para calcular a função sigmoide e backpropagation. É notório lembrar que realizasse os cálculos mesmo com ordem invertida, onde o objeto Valor pode ser o primeiro ou segundo operando.

In [9]:
import math

class Valor:
    """
    Classe que representa um valor numérico com suporte a diferenciação automática.
    
    Cada instância armazena:
      - data: o valor numérico.
      - progenitor: tupla com os valores dos quais este foi derivado.
      - operador_mae: string representando a operação que gerou o valor.
      - grad: gradiente (inicialmente zero) usado no backpropagation.
    """
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        """
        Inicializa uma instância de Valor.
        
        Args:
            data (float): o valor numérico.
            progenitor (tuple): valores anteriores que contribuíram para este.
            operador_mae (str): operação que gerou o valor.
            rotulo (str): rótulo opcional para identificação.
        """
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        """Retorna uma representação string simplificada do objeto."""
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        """
        Sobrecarga do operador de adição.
        
        Realiza a operação: self + outro_valor.
        
        Args:
            outro_valor (Valor ou número): valor a ser somado.
        
        Returns:
            Valor: novo objeto representando a soma.
        """
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado

    def __mul__(self, outro_valor):
        """
        Sobrecarga do operador de multiplicação.
        
        Realiza a operação: self * outro_valor.
        
        Args:
            outro_valor (Valor ou número): valor a ser multiplicado.
        
        Returns:
            Valor: novo objeto representando o produto.
        """
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado

    def exp(self):
        """
        Calcula a exponencial do valor.
        
        Realiza a operação: exp(self).
        
        Returns:
            Valor: novo objeto representando a exponencial.
        """
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado

    def __pow__(self, expoente):
        """
        Sobrecarga do operador de exponenciação.
        
        Realiza a operação: self ** expoente.
        
        Args:
            expoente (int ou float): expoente da operação.
        
        Returns:
            Valor: novo objeto representando a exponenciação.
        """
        assert isinstance(expoente, (int, float)), "Expoente deve ser um número."
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado

    def __truediv__(self, outro_valor):
        """
        Sobrecarga do operador de divisão.
        
        Realiza a operação: self / outro_valor.
        
        Args:
            outro_valor (Valor ou número): divisor.
        
        Returns:
            Valor: novo objeto representando a divisão.
        """
        return self * outro_valor ** (-1)

    def __neg__(self):
        """
        Sobrecarga do operador de negação.
        
        Realiza a operação: -self.
        
        Returns:
            Valor: novo objeto representando o valor negativo.
        """
        return self * -1

    def __sub__(self, outro_valor):
        """
        Sobrecarga do operador de subtração.
        
        Realiza a operação: self - outro_valor.
        
        Args:
            outro_valor (Valor ou número): valor a ser subtraído.
        
        Returns:
            Valor: novo objeto representando a subtração.
        """
        return self + (-outro_valor)

    def __radd__(self, outro_valor):
        """
        Sobrecarga do operador de adição reversa.
        
        Permite operações onde Valor está à direita: outro_valor + self.
        
        Args:
            outro_valor (Valor ou número): valor a ser somado.
        
        Returns:
            Valor: resultado da adição.
        """
        return self + outro_valor

    def __rmul__(self, outro_valor):
        """
        Sobrecarga do operador de multiplicação reversa.
        
        Permite operações onde Valor está à direita: outro_valor * self.
        
        Args:
            outro_valor (Valor ou número): valor a ser multiplicado.
        
        Returns:
            Valor: resultado da multiplicação.
        """
        return self * outro_valor

    def sig(self):
        """
        Calcula a função sigmoide.
        
        Realiza a operação: exp(self) / (exp(self) + 1).
        
        Returns:
            Valor: novo objeto representando o resultado da sigmoide.
        """
        return self.exp() / (self.exp() + 1)

    def propagar(self):
        """
        Função de propagação (backpropagation) do gradiente.
        
        Este método deve ser sobrescrito pelas operações específicas.
        """
        pass

    def propagar_tudo(self):
        """
        Executa o backpropagation através de todos os nós (valores) conectados.
        
        Atribui gradiente 1 ao vértice folha e propaga recursivamente utilizando uma ordem topológica dos nós.
        """
        self.grad = 1
        ordem_topologica = []
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)

        for vertice in reversed(ordem_topologica):
            vertice.propagar()


Para termos o dataset e poder realizar as avaliações da nossa rede neural, bem como realizar alguns cálculos matemáticos, vamos importar as bibliotecas necessárias e carregar o dataset. 

In [10]:
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

O dataset Iris contém informações sobre três espécies de flores: setosa, versicolor e virginica. Cada amostra possui quatro atributos: comprimento e largura das sépalas e pétalas.

In [11]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


Vamos transformar as classes em valores numéricos e dividir os dados em conjuntos de treino e teste.

In [12]:
le = LabelEncoder()
iris['species'] = le.fit_transform(iris['species'])

X = iris.drop('species', axis=1).values
y = iris['species'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Vamos usar a implementação de MLP já existente no notebook para criar e treinar a rede neural.

In [13]:
NUM_DADOS_DE_ENTRADA = 4
NUM_DADOS_DE_SAIDA = 3
CAMADAS_OCULTAS = [5, 5]

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

minha_mlp = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

NameError: name 'MLP' is not defined

Vamos treinar a rede neural usando o conjunto de treino e calcular a perda em cada época.

In [ ]:
NUM_EPOCAS = 200
TAXA_DE_APRENDIZADO = 0.01

for epoca in range(NUM_EPOCAS):
    y_pred = []
    for exemplo in X_train:
        previsao = minha_mlp(exemplo)
        y_pred.append(previsao)

    erros = []
    for yt, yp in zip(y_train, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo**2
        erros.append(erro_quadratico)
    loss = sum(erros)

    for p in minha_mlp.parametros():
        p.grad = 0

    loss.propagar_tudo()

    for p in minha_mlp.parametros():
        p.data = p.data - p.grad * TAXA_DE_APRENDIZADO

    if epoca % 10 == 0:
        print(f"Época {epoca}, Perda: {loss.data}")

## Avaliação da Rede Neural

Vamos avaliar o desempenho da rede neural no conjunto de teste.

## 🤓 Neurônio

A classe Neurônio representa um neurônio em uma rede neural. Ela possui pesos e um viés, que são inicializados aleatoriamente. O neurônio calcula a saída usando a função sigmoide e realiza o backpropagation para atualizar os pesos e o viés com base no erro da previsão.

In [ ]:
import random


class Neuronio:
    """
    Representa um neurônio simples com pesos e viés para uso em uma rede neural.

    Este neurônio utiliza a classe Valor para armazenar seus parâmetros e realizar a
    diferenciação automática durante o treinamento.
    """

    def __init__(self, num_dados_entrada):
        """
        Inicializa um neurônio com um número especificado de entradas.

        Args:
            num_dados_entrada (int): número de entradas para o neurônio.
        """
        self.vies = Valor(random.uniform(-1, 1))

        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))

    def __call__(self, x):
        """
        Realiza a passagem forward do neurônio.

        Calcula a soma ponderada das entradas e aplica a função sigmoide para
        determinar a saída do neurônio.

        Args:
            x (list[Valor]): lista de objetos Valor representando as entradas.

        Returns:
            Valor: objeto Valor representando a saída do neurônio.
        """
        assert len(x) == len(
            self.pesos
        ), "O número de entradas deve ser igual ao número de pesos."

        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno

        soma += self.vies

        dado_de_saida = soma.sig()

        return dado_de_saida

    def parametros(self):
        """
        Retorna uma lista com os parâmetros do neurônio (pesos e viés).

        Returns:
            list[Valor]: lista contendo os pesos e o viés.
        """
        return self.pesos + [self.vies]

## 🎂 Camada

A classe Camada representa uma camada de neurônios em uma rede neural. Ela possui um número específico de neurônios e é responsável por calcular a saída da camada com base nas entradas recebidas. Realiza o forward pass e agrega os parametros de cada neurônio.

In [ ]:
class Camada:
    """
    Representa uma camada em uma rede neural composta por múltiplos neurônios.

    Cada camada gerencia um conjunto de neurônios, realizando a passagem forward
    e agregando os parâmetros (pesos e viés) de cada neurônio.
    """

    def __init__(self, num_neuronios, num_dados_entrada):
        """
        Inicializa a camada com um número específico de neurônios, cada um com
        um determinado número de entradas.

        Args:
            num_neuronios (int): número de neurônios na camada.
            num_dados_entrada (int): número de entradas para cada neurônio.
        """
        self.neuronios = []
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            self.neuronios.append(neuronio)

    def __call__(self, x):
        """
        Realiza a passagem forward na camada.

        Aplica cada neurônio da camada à mesma entrada e retorna os dados de saída.

        Args:
            x (list[Valor]): lista de objetos Valor representando as entradas da camada.

        Returns:
            Valor ou list[Valor]: saída de um único neurônio se houver apenas um,
            ou lista com as saídas de todos os neurônios.
        """
        dados_de_saida = []
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)

        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:
            return dados_de_saida

    def parametros(self):
        """
        Agrega e retorna todos os parâmetros (pesos e viés) de cada neurônio da camada.

        Returns:
            list[Valor]: lista contendo todos os parâmetros dos neurônios da camada.
        """
        params = []
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)

        return params

## 🧠 MLP

Por fim, nossa última classe da nossa rede neural é a `MLP` (Multi-Layer Perceptron). Ela representa uma rede neural com múltiplas camadas. A `MLP` organiza as camadas da rede, permitindo a passagem forward dos dados e a agregação dos parâmetros de todas as camadas.

In [ ]:
class MLP:
    """
    Representa uma rede neural do tipo MLP (Multi-Layer Perceptron).

    Essa classe organiza as camadas da rede, permitindo a passagem forward dos dados e
    a agregação dos parâmetros (pesos e viés) de todas as camadas.
    """

    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        """
        Inicializa a MLP com um número definido de entradas e uma lista que especifica
        o número de neurônios em cada camada.

        Args:
            num_dados_entrada (int): número de entradas da rede.
            num_neuronios_por_camada (list[int]): lista com o número de neurônios para cada camada.
        """
        percurso = [num_dados_entrada] + num_neuronios_por_camada

        camadas = []
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)

        self.camadas = camadas

    def __call__(self, x):
        """
        Realiza a passagem forward pela rede.

        Cada camada processa a entrada e o resultado é passado para a próxima camada.

        Args:
            x (list[Valor] ou Valor): dados de entrada para a rede.

        Returns:
            Valor ou list[Valor]: saída final da rede após a passagem por todas as camadas.
        """
        for camada in self.camadas:
            x = camada(x)
        return x

    def parametros(self):
        """
        Agrega e retorna todos os parâmetros (pesos e viés) de todas as camadas da rede.

        Returns:
            list[Valor]: lista contendo os parâmetros de cada camada.
        """
        params = []
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)

        return params

In [ ]:
# Avaliação
y_pred_test = []
for exemplo in X_test:
    previsao = minha_mlp(exemplo)
    y_pred_test.append(np.argmax(previsao))

# Calculando acurácia
acuracia = np.mean(y_pred_test == y_test)
print(f'Acurácia no conjunto de teste: {acuracia * 100:.2f}%')

# 📖 Referências

[1] Conjunto de dados Iris. UCI Machine Learning Repository. Disponível em: https://archive.ics.uci.edu/dataset/53/iris. Acesso em: 05 de Abril de 2025.

[2] Conjunto de dados Iris. Wikipedia. Disponível em: https://pt.wikipedia.org/wiki/Conjunto_de_dados_flor_Iris. Acesso em: 05 de Abril de 2025.